# Importando Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from pandas import DataFrame as df
from scipy.stats import zscore
import seaborn as sns
import re
import os

from ydata_profiling import ProfileReport
import sweetviz as sv

# Carregando e inspecionando dados

In [ ]:
main_dataframe = pd.read_csv(
  "./data/card_transdata.csv", 
  na_values=[' ', '_', '_______'], # substituir com NaN do dataset
  low_memory=False
)

# Mostrando as primeiras 5 linhas do Dataframe Original

In [ ]:
main_dataframe.head()

# Mostrando o formato do Dataframe Original


In [ ]:
# número de colunas
print(f'columns: {main_dataframe.shape[1]}')

# número de linhas
print(f'rows: {main_dataframe.shape[0]}')

# Obtendo informações sobre cada variável do dataframe original

In [ ]:
main_dataframe.info()

# Remoção dos valores FRAUDES != True

In [ ]:
# Remover todas as linhas onde 'fraud' é igual a 0
fraud_dataframe = main_dataframe[main_dataframe['fraud'] != 0]

# Redefinir o índice do DataFrame
fraud_dataframe.reset_index(drop=True, inplace=True)
fraud_dataframe = fraud_dataframe.drop('fraud', axis=1) # Remover a coluna 'fraud'

# Apresentando o Novo Dataframe SOMENTE com FRAUDES == True

In [ ]:
fraud_dataframe

# Mostra as primeiras 5 linhas do Novo Dataframe SOMENTE com FRAUDES == True

In [ ]:
fraud_dataframe.head()

# Obtendo informações sobre cada variável do Novo Dataframe SOMENTE com FRAUDES == True

In [ ]:
fraud_dataframe.info()

# Mostrando o formato do Novo Dataframe SOMENTE com FRAUDES == True

In [ ]:
# número de colunas
print(f'columns: {fraud_dataframe.shape[1]}')

# número de linhas
print(f'rows: {fraud_dataframe.shape[0]}')

# Limpeza de dados

### 1. Valores ausentes

In [ ]:
# Verifique a soma dos valores ausentes para cada coluna
fraud_dataframe.isnull().sum()

### 2. Valores duplicados

In [ ]:
# Verifique a soma das linhas duplicadas
print(f'duplicate rows: {fraud_dataframe.duplicated().sum()}')

# Algumas informações estatísticas

In [ ]:
# Criando uma tabela de estatísticas descritivas para dados
fraud_dataframe.describe().T

# Análise Exploratória dos Dados (EDA)

## Recursos Categóricos

### Distribuição de Compras em Lojas Repetidas

In [ ]:
# Contagem de valores únicos na coluna 'repeat_retailer'
repeat_retailer_count = fraud_dataframe['repeat_retailer'].value_counts()
repeat_retailer_count

In [ ]:
# Configurando o estilo do Seaborn
sns.set(style="whitegrid")

# Plotagem do gráfico de torta apenas para 'repeat_retailer' 
plt.figure(figsize=(6, 6))
plt.pie(repeat_retailer_count.values, labels=repeat_retailer_count.index, autopct='%1.1f%%')
plt.title('Distribuição das Compras em Lojas Repetidadas')

# Adicionando legendas
legend_labels = ['1 == Repetidas', '0 == Não Repetidas']
plt.legend(legend_labels, loc='lower right')

plt.show()

### Distribuição de Compras com Uso do Chip

In [ ]:
# Contagem de valores únicos na coluna 'used_chip'
used_chip_count = fraud_dataframe['used_chip'].value_counts()
used_chip_count

In [ ]:
# Configurando o estilo do Seaborn
sns.set(style="whitegrid")

# Plotagem do gráfico de torta apenas para 'used_chip' 
plt.figure(figsize=(6, 6))
plt.pie(used_chip_count.values, labels=used_chip_count.index, autopct='%1.1f%%')
plt.title('Distribuição das Compras com Uso do Chip do Cartão')

# Adicionando legendas
legend_labels = ['0 == Sem chip', '1 == Com chip']
plt.legend(legend_labels, loc='lower right')

plt.show()

### Distribuição de Compras com Uso da Senha

In [ ]:
# Contagem de valores únicos na coluna 'used_pin_number'
used_pin_number_count = fraud_dataframe['used_pin_number'].value_counts()
used_pin_number_count

In [ ]:
# Configurando o estilo do Seaborn
sns.set(style="whitegrid")

# Plotagem do gráfico de torta apenas para 'used_pin_number_count' 
plt.figure(figsize=(6, 6))
plt.pie(used_pin_number_count.values, labels=used_pin_number_count.index, autopct='%1.1f%%')
plt.title('Distribuição das Compras com Uso da Senha')

# Adicionando legendas
legend_labels = ['0 == Sem Senha', '1 == Com Senha']
plt.legend(legend_labels, loc='lower right')

plt.show()

### Distribuição de Compras Onlines

In [ ]:
# Contagem de valores únicos na coluna 'online_order'
online_order_count = fraud_dataframe['online_order'].value_counts()
online_order_count

In [ ]:
# Configurando o estilo do Seaborn
sns.set(style="whitegrid")

# Plotagem do gráfico de torta apenas para 'online_order_count' 
plt.figure(figsize=(6, 6))
plt.pie(online_order_count.values, labels=online_order_count.index, autopct='%1.1f%%')
plt.title('Distribuição das compras em Lojas (Físicas e Onlines) ')

# Adicionando legendas
legend_labels = ['1 == Loja Online', '0 == Loja Física']
plt.legend(legend_labels, loc='lower right')

plt.show()

### Distribuição de todas colunas categóricas

In [ ]:
# Crie uma lista contendo recursos categóricos
category_features = ['repeat_retailer', 'used_chip','used_pin_number', 'online_order']

In [ ]:
# Configurando o estilo do Seaborn
sns.set(style="whitegrid")

# Desenhando um gráfico de pizza para exibir a distribuição de cada coluna categórica
plt.figure(figsize = (16, 12))
for i, col in enumerate(category_features):
    plt.subplot(1, 4, i+1)
    plt.pie(x = fraud_dataframe[col].value_counts().values, 
            labels = fraud_dataframe[col].value_counts().index, 
            autopct = '%1.1f%%')
    plt.xlabel(col, weight = 'bold')
plt.show()

## Recursos Numéricos

### Distribuição das Distâncias de Casa

In [ ]:
# Configurando o estilo do Seaborn
sns.set(style="whitegrid")

# Plotagem do boxplot para 'distance_from_home'
plt.figure(figsize=(8, 6))
sns.boxplot(data=fraud_dataframe, y='distance_from_home', color='skyblue')
plt.title('Boxplot da Distância de Casa')
plt.ylabel('Distância de Casa')

plt.show()

### Distribuição das Distâncias da Última da Transação

In [ ]:
# Configurando o estilo do Seaborn
sns.set(style="whitegrid")

# Plotagem do boxplot para 'distance_from_last_transaction'
plt.figure(figsize=(8, 6))
sns.boxplot(data=fraud_dataframe, y='distance_from_last_transaction', color='skyblue')
plt.title('Boxplot da Distância da Última Transação')
plt.ylabel('Distância da Última Transação')

plt.show()

### Distribuição da Proporção entre a transação do preço de compra e o preço de compra médio

In [ ]:
# Configurando o estilo do Seaborn
sns.set(style="whitegrid")

# Plotagem do boxplot para 'ratio_to_median_purchase_price'
plt.figure(figsize=(10, 8))
sns.boxplot(data=fraud_dataframe, y='ratio_to_median_purchase_price', color='skyblue')
plt.title('Boxplot da Proporção entre a transação do preço de compra e o preço de compra médio.')
plt.ylabel('Proporção entre a transação do preço de compra e o preço de compra médio.')

plt.show()

In [ ]:
# Crie uma lista contendo recursos numéricos
num_features = ['distance_from_home', 'distance_from_last_transaction', 'ratio_to_median_purchase_price']

In [ ]:
# Drawing a boxplot to display the distribution of each numerical column to detect the outliers
plt.figure(figsize = (18, 4))
for i, col in enumerate(num_features):
    plt.subplot(1, 3, i+1)
    sns.boxplot(data = main_dataframe, x = col)
plt.show()

In [ ]:
analise = sv.analyze(fraud_dataframe)
analise.show_html('sweetviz.html')

prof = ProfileReport(fraud_dataframe)
prof.to_file(output_file='pandasprof.html')

# KMEANS Algorithm

### Definicoes

In [ ]:
n_broad_clusters = range(2, 15)

# treinamento KMeans
max_iter = 10_000
n_init = 100, 
random_state = 61658

n_clusters = 4

## Preparação dos dados para aplicação do KMEANS

In [ ]:
grouped_mean_dataframe = fraud_dataframe.div(fraud_dataframe.sum(axis=1), axis='rows')
grouped_mean_dataframe

In [ ]:
clusters_distortion = []
for n_clus in n_broad_clusters:
    print(n_clus)
    clusters_distortion.append(KMeans(n_clusters=n_clus, max_iter=max_iter, n_init=n_init, random_state=random_state).fit(grouped_mean_dataframe).inertia_ )

In [ ]:
clusters_distortion

## Método do Cotovelo

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(n_broad_clusters, clusters_distortion)
plt.grid()

## Analise de clusters

In [ ]:
km = KMeans(n_clusters=n_clusters, max_iter=max_iter, n_init=n_init, random_state=random_state)
clusters_classes = km.fit_predict(grouped_mean_dataframe)
clusters_classes

In [ ]:
fraud_dataframe.loc[:, 'clus'] = x_t
fraud_dataframe

In [ ]:
for col in fraud_dataframe.drop( 'clus', axis=1) .columns: 
    for cl in np.sort(fraud_dataframe.clus.unique()):
        if fraud_dataframe.dtypes[col] == object:
            vc = 100*fraud_dataframe.loc[fraud_dataframe.clus==cl, col].value_counts() / (fraud_dataframe.clus==cl) . sum()
            for cat, cnt in vc.reset_index().values:
                print(f'{cl:d}; {col}_{str(cat):5}; {cnt:5.2f}%'.replace('.', ','))
        else:
            print(f'{cl:d}; {col}; {fraud_dataframe.loc[fraud_dataframe.clus==cl, col].mean(): .2f}'.replace('.', ','))

for col in fraud_dataframe.drop( 'clus', axis=1).columns:
    if fraud_dataframe.dtypes[col] == object:
        vc = 100*fraud_dataframe.loc[:, col].value_counts() / (fraud_dataframe.shape[0])
        for cat, cnt in vc.reset_index().values:
            print(f'All; {col}_{str(cat):s}; {cnt:5.2f}%'.replace('.', ','))
    else:
        print(f'All; {col}; {fraud_dataframe.loc[:, col].mean():.2f}'.replace('.', ','))

In [ ]:
pca = PCA(n_components=2)
main_components = pca.fit_transform(fraud_dataframe)

In [ ]:
# Agora, você tem os resultados do KMeans (clusters) e os principais componentes após a redução de dimensionalidade com PCA
# Vamos plotar os resultados em um gráfico de dispersão

plt.figure(figsize=(10, 6))

# Plot os pontos dos dados após a redução de dimensionalidade, coloridos pelos clusters atribuídos pelo KMeans
for i in range(n_clusters):
    plt.scatter(main_components[x_t == i, 0], main_components[x_t == i, 1], label=f'Cluster {i}')

plt.title('Resultados do KMeans com PCA para Bidimensionalização')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))

for i in range(n_clusters):
    plt.figure(figsize=(8, 6))
    cluster_points = main_components[x_t == i]
    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], label=f'Cluster {i}')
    plt.title(f'Cluster {i}')
    plt.xlabel('Componente Principal 1')
    plt.ylabel('Componente Principal 2')
    plt.legend()
    plt.grid(True)
    plt.show()

pca - reduzir dimensionalidade  \/ mapa de calor com classes